In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import os

# Configurações
model_name = "BAAI/bge-m3"
output_folder = 'vectorstore'
existing_db_path = 'vectorstore/db_faiss_all_pdfs'


def load_and_process_pdfs(pdf_folder):
    documents = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            file_path = os.path.join(pdf_folder, filename)
            loader = PyPDFLoader(file_path)
            documents.extend(loader.load())
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    split_docs = text_splitter.split_documents(documents)
    return split_docs


def get_vectorstore(pdf_folder):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    if os.path.exists(existing_db_path):
        return FAISS.load_local(existing_db_path, embeddings, allow_dangerous_deserialization=True)
    else:
        docs = load_and_process_pdfs(pdf_folder)
        vectorstore = FAISS.from_documents(docs, embeddings)
        vectorstore.save_local(existing_db_path)
        return vectorstore


vectorstore = get_vectorstore('pdf')
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 5})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [2]:
def model_ollama(model="llama3.2", temperature=0.1):
    return ChatOllama(model=model, temperature=temperature)

llm = model_ollama()

In [3]:
context_q_system_prompt = """
Dado o histórico de chat e a pergunta de acompanhamento, que pode fazer referência ao contexto no histórico do chat, 
formule uma pergunta independente que possa ser entendida sem o histórico do chat. 
NÃO responda à pergunta, apenas reformule-a se necessário e, caso contrário, retorne-a como está.
"""

qa_prompt_template = """
Você é um assistente virtual especializado em analisar e responder perguntas sobre medicamentos.
Use os seguintes trechos de contexto recuperado para responder à pergunta.
Se você não souber a resposta, diga honestamente que não sabe. Mantenha a resposta concisa e focada.
Se for solicitado a listar as referências dos artigos, leias, informações específicas do documento, faça-o de forma estruturada e clara.
Responda em português.

Pergunta: {input}

Contexto: {context}

Resposta:
"""

context_q_prompt = ChatPromptTemplate.from_messages([
    ("system", context_q_system_prompt),
    ("human", "Pergunta: {input}")
])

history_aware_retriever = create_history_aware_retriever(llm=llm, retriever=retriever, prompt=context_q_prompt)

qa_prompt = PromptTemplate.from_template(qa_prompt_template)
qa_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

# Função para fazer perguntas
def ask_question(question, chat_history=''): 
    result = rag_chain.invoke({"input": question, "chat_history": chat_history})
    return result['answer']

In [6]:
user_query = 'De acordo com a bula, qual a posologia da dipirona? Não especifique o público alvo'
response = ask_question(user_query)
print(response)

A posologia da dipirona é a seguinte:

* Adultos e adolescentes acima de 15 anos: 20 a 40 gotas em administração única ou até o máximo de 40 gotas, 4 vezes ao dia.
* Crianças: devem receber dipirona monoidratada solução oral (gotas) conforme seu peso seguindo a orientação do esquema abaixo:
 + Peso: 
 - Até 10 kg: 5-10 gotas
 - 11-20 kg: 10-15 gotas
 - 21-30 kg: 15-20 gotas
 - Acima de 30 kg: 20-25 gotas

É importante notar que a administração de dipirona deve ser feita com cuidado em pacientes com condições específicas, como asma brônquica, urticária crônica, intolerância ao álcool e intolerância a corantes ou conservantes. Além disso, é recomendada supervisão médica em pacientes idosos, crianças pequenas e outros grupos de risco.


In [5]:
user_query = """Extrair posologias indicadas segundo a bula da dipirona"""
response = ask_question(user_query)
print(response)

Segundo a bula da dipirona, as posologias indicadas são:

* Adultos e adolescentes acima de 15 anos: 20 a 40 gotas em administração única ou até o máximo de 40 gotas, 4 vezes ao dia.
* Crianças: recebem dipirona monoidratada solução oral (gotas) conforme seu peso seguindo um esquema específico.

É importante notar que a dose e a via de administração dependem do efeito analgésico desejado e das condições do paciente. Em alguns casos, a administração por via intravenosa ou intramuscular pode ser necessária. Além disso, o tratamento pode ser interrompido a qualquer instante sem provocar danos ao paciente.

Não há informações sobre a dose para pacientes com idade inferior a 15 anos ou com condições específicas que possam afetar a posologia da dipirona.

Referências:

* Bula da dipirona (não especificada)
* Wong et al. (2001) - não especificado
* Izhar T. (1999) - não especificado
